In [1]:
# сделаем необходимые импорты

In [2]:
from dash import Dash, html, dcc, Input, Output
import dash_bootstrap_components as dbc

import pandas as pd
import plotly.express as px

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [3]:
# построим дэшборд, в котором будут аннотация, 2 выпадающих списка и 3 графика
# 1ый график будет зависеть от 1го параметра, отражает показатели в выбраном городе за неделю
# 2ой и 3ий от 2х параметров, отражают показатели в выбраном городе за сутки

In [4]:
app = Dash()
server = app.server


df = pd.read_csv('output.csv')
days_w = ['Понедельник','Вторник','Среда','Четверг','Пятница','Суббота','Воскресенье']


app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])

info_content = ['Данный дэшборд визуализирует музыкальные предпочтения пользователей Яндекс Музыки в 5 крупнейших городах РФ за неделю.',
                html.Br(),
                'Ссылка на проект, в котором происходят первичные действия и заготовки, находится ниже.',
               html.Br(), html.Br(),
               html.A('Ссылка на проект', href = '', target = '_blank'),
               html.Br(), html.Br()]

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1('Аналитика по музыкальным запросам за неделю')], 
                     width = {'size':12, 'offset':0})]), 
    html.Br(),
    dbc.Row([
        dbc.Col([
            html.P(info_content)], 
                     width=12)]), 
    html.Br(),    
    dbc.Row([
        dbc.Col([
            html.Label('Город'),dcc.Dropdown(options = [{'label':d, 'value':d} for d in sorted(df['City'].unique())], 
                                                       value = 'Москва', 
                                                       id = 'city', 
                                             clearable = False)], 
                     width=6), 
    dbc.Col([
        html.Label('День недели'),dcc.Dropdown(options = [{'label':d, 'value':d} for d in days_w], 
                                                    value = 'Понедельник', 
                                                    id = 'day', 
                                               clearable = False)], width=6)]),
    html.Br(),
    dbc.Row([
        dbc.Col([
            dcc.Graph(id = 'graph1', figure = {})])]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(id = 'graph2', figure = {})], width = {'size':12, 'offset':0})]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(id = 'graph3', figure = {})])])
])   

    
@app.callback(Output('graph1', 'figure'), Input('city', 'value'))
def update_figure1(city):
    print(city)
    
    week = df.groupby('City')['Day'].value_counts().to_frame()
    week.rename(columns={'Day': 'Values'}, inplace=True)
    week.reset_index(inplace = True)
    
    figure = px.bar(week[week['City'] == city].sort_values('Values'), 
           x = 'Day', 
           y = 'Values', 
           color = 'Values', title = 'Количество мызыкальных запросов за неделю ({0})'.format(city),
           labels={'Values': 'Запросы', 'Day': 'День'}, text_auto = True).update_layout(bargap=0.07, 
                                           xaxis_title='День недели', 
                                           yaxis_title = 'Количество запросов', 
                                           template='plotly_dark',
                                           coloraxis=dict(colorscale='Bluered_r'))
    return figure
    

    
@app.callback(Output('graph2', 'figure'), Output('graph3', 'figure'), Input('city', 'value'), Input('day', 'value'))
def update_figure2(city, weekday):
    print(city,weekday)
    morning = df[(df['City'] == city) 
                 & (df['Day'] == weekday) 
                 & (df['Day_period'] == 'утро')]['Genre'].value_counts().nlargest(10)
    day = df[(df['City'] == city) 
             & (df['Day'] == weekday) 
             & (df['Day_period'] == 'день')]['Genre'].value_counts().nlargest(10)
    evening = df[(df['City'] == city) 
                 & (df['Day'] == weekday) & (df['Day_period'] == 'вечер')]['Genre'].value_counts().nlargest(10)
    
    fig = make_subplots(rows = 1, cols = 3, subplot_titles = ('Утро', 'День', 'Вечер'))


    fig.add_trace(go.Bar(x = [m for m in morning.index], 
                         y = [g for g in morning.values], 
                         name = 'Утро',
                         text = [g for g in morning.values]), row=1, col=1)
    fig.add_trace(go.Bar(x = [m for m in day.index], 
                         y = [g for g in day.values], 
                         name = 'День',
                         text = [g for g in day.values]), row=1, col=2)
    fig.add_trace(go.Bar(x = [m for m in evening.index], 
                         y = [g for g in evening.values], 
                         name = 'Вечер',
                         text = [g for g in evening.values]), row=1, col=3)

    
    fig.update_xaxes(title_text = 'Жанр', row=1, col=1)
    fig.update_xaxes(title_text = 'Жанр', row=1, col=2)
    fig.update_xaxes(title_text = 'Жанр', row=1, col=3)

    fig.update_yaxes(title_text = 'Количество запросов', row=1, col=1)
    fig.update_yaxes(title_text = 'Количество запросов', row=1, col=2)
    fig.update_yaxes(title_text = 'Количество запросов', row=1, col=3)

    fig.update_layout(template='plotly_dark')

    fig.update_layout(title_text = 'Топ 10 музыкальных жанров ({0}, {1})'.format(city, weekday))
    
    
    genre = df[(df['City'] == city) & (df['Day'] == weekday)]['Genre'].value_counts()
    
    fig2 = px.bar(genre.sort_values(), 
       color = 'value',
       orientation ='h', 
       text_auto = True,
       labels={'value': 'Запросы', 'index': 'Жанр'},
       title ='Популярность музыкальных жанров ({0}, {1})'.format(city, weekday)).update_layout(                                                                      height=2000, 
                                                                                      bargap=0.07, 
                                                                                        template='plotly_dark',
                                                                                      xaxis_title='Количество запросов', 
                                                                                      yaxis_title = 'Музыкальные жанры')
    
    
    return fig, fig2


if __name__=='__main__':
    app.run_server(port=8042)